In [ ]:
import json
import networkx as nx
from networkx.readwrite import node_link_graph

def read_json_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        data = json.load(f)
    return data

data = read_json_file('MC1_graph.json')

# Convert the JSON data to a NetworkX graph
G = node_link_graph(data)
nodes = G.nodes()
edges = G.edges()

print(f"节点数: {G.number_of_nodes()}")
print(f"边数: {G.number_of_edges()}")

节点数: 17412
边数: 37857


## 1. 对人、乐队、音乐公司节点计算其影响力分数

$$ 
score_i = 2\times \text{number of notable work} + d_i \\
\text{where} \quad d_i\quad \text{means degree,即节点度数，该节点连接几条边} 

In [27]:
for node in nodes:
    for edge in list(G.in_edges(node, keys = True)):
        print(f"节点 {node} 的边: {edge}")
    break

节点 0 的边: (1, 0, 0)
节点 0 的边: (2, 0, 0)
节点 0 的边: (3, 0, 0)


In [ ]:
for node_id in G.nodes:
    if G.nodes[node_id].get('Node Type') in ['Person', 'MusicalGroup']: # RecordLabel后面单独计算
        influence_socre = G.degree(node_id) # 先加上度数
        for edge in G.out_edges(node_id):   # 以person为源节点
            if G[edge[1]].get('Node Type') in ['Song', 'Album']: # 确保是歌曲或专辑
                if G[edge[1]].get('notable'):
                    influence_socre += 2
        G.nodes[node_id]['influence_score'] = influence_socre
#print(G.nodes(data=True))
influence_socres = [G.nodes[node_id]['influence_score'] for node_id in G.nodes if 'influence_score' in G.nodes[node_id]]
print(f"影响力分数: {influence_socres}")
print(f"平均影响力分数: {sum(influence_socres) / len(influence_socres)}")

影响力分数: [2, 2, 1, 1, 12, 4, 3, 2, 4, 3, 2, 1, 2, 3, 4, 1, 9, 33, 1, 4, 5, 6, 2, 2, 1, 2, 1, 6, 2, 6, 2, 1, 1, 1, 1, 6, 1, 6, 2, 1, 8, 7, 6, 7, 5, 2, 1, 3, 7, 3, 1, 1, 1, 1, 3, 8, 2, 2, 5, 6, 10, 3, 3, 19, 3, 1, 1, 1, 1, 5, 1, 1, 1, 1, 1, 1, 1, 1, 1, 15, 9, 6, 2, 5, 9, 16, 13, 2, 2, 3, 1, 3, 2, 8, 18, 5, 3, 14, 1, 4, 1, 1, 1, 1, 1, 2, 2, 12, 2, 2, 4, 1, 1, 3, 3, 1, 1, 2, 1, 5, 1, 2, 1, 2, 1, 8, 2, 4, 8, 16, 3, 2, 1, 1, 1, 3, 12, 13, 1, 4, 2, 1, 18, 2, 3, 2, 3, 1, 1, 1, 1, 3, 6, 2, 3, 3, 4, 1, 2, 2, 1, 1, 1, 1, 1, 4, 2, 1, 1, 1, 7, 3, 6, 2, 9, 15, 19, 18, 2, 1, 1, 11, 3, 7, 13, 1, 2, 1, 1, 1, 3, 2, 6, 3, 6, 1, 5, 1, 2, 4, 1, 1, 2, 2, 1, 10, 1, 2, 4, 4, 2, 2, 1, 2, 2, 5, 1, 1, 2, 3, 2, 3, 1, 1, 1, 1, 1, 2, 2, 1, 1, 1, 3, 1, 1, 6, 3, 28, 2, 1, 3, 2, 3, 1, 2, 3, 8, 1, 3, 1, 2, 4, 3, 2, 1, 7, 9, 5, 8, 2, 1, 8, 4, 1, 13, 5, 8, 2, 1, 1, 1, 1, 2, 1, 1, 1, 1, 2, 1, 6, 4, 1, 1, 5, 1, 1, 1, 5, 1, 1, 2, 1, 1, 2, 1, 4, 4, 2, 2, 2, 1, 2, 1, 1, 4, 1, 1, 4, 1, 2, 1, 2, 2, 2, 5, 1, 5, 6, 5, 49, 1, 2, 1, 

In [ ]:
# 计算RecordLabel的影响力分数
for node_id in G.nodes:
    if G.nodes[node_id].get('Node Type') == 'RecordLabel':
        influence_socre = G.degree(node_id) # 先加上度数
        for edge in G.in_edges(node_id):    # 以RecordLabel为目标节点,则源节点只能是歌曲专辑
            if G[edge[0]].get('notable'):
                influence_score += 2
        for edge in G.out_edges(node_id):   # 以RecordLabel为源节点
            if G[edge[1]].get('Node Type') in ['Song', 'Album']:
                if G[edge[1]].get('notable'):
                    influence_socre += 2
        print(f"RecordLabel {node_id} 的影响力分数: {influence_socre}")
        G.nodes[node_id]['influence_score'] = influence_socre


## 对Person 或 MusicalGroup 节点，根据其相关作品计算得出其主导流派。

In [ ]:
for node_id in G.nodes:
    if G.nodes[node_id].get('Node Type') in ['Person', 'MusicalGroup']:
        genres = {}
        for edge in G.out_edges(node_id, keys = True):    # 以Person或MusicalGroup为源节点
            if G.edges[edge].get('Edge Type') in ['PerformerOf', 'ComposerOf', 'LyricistOf', 'ProducerOf']:
                #print(1)
                if G.nodes[edge[1]].get('Node Type') in ['Song', 'Album']:
                    #print(2)
                    genre = G.nodes[edge[1]].get('genre')
                    genres[genre] = genres.get(genre, 0) + 1
        if genres:
            max_genre = max(genres, key=genres.get)
            #print(f"节点 {node_id} 的最大流派: {max_genre}")
            G.nodes[node_id]['max_genre'] = max_genre
        else:
            print(f"节点 {node_id} 没有流派信息")
            G.nodes[node_id]['max_genre'] = None

# 节点17107-17109同属于乐队17110，因此流派信息等同于17110
G.modes[17107]['max_genre'] = G.nodes[17110]['max_genre']
G.modes[17108]['max_genre'] = G.nodes[17110]['max_genre']
G.modes[17109]['max_genre'] = G.nodes[17110]['max_genre']
                    

节点 17107 没有流派信息
节点 17108 没有流派信息
节点 17109 没有流派信息


## 如果 Person 或 MusicalGroup 节点创作了大量知名作品，标记为notable

In [61]:
for node_id in G.nodes:
    if G.nodes[node_id].get('Node Type') in ['Person', 'MusicalGroup']:
        notable_work_num = 0
        for edge in G.out_edges(node_id, keys = True):    # 以Person或MusicalGroup为源节点
            if G.edges[edge].get('Edge Type') in ['PerformerOf', 'ComposerOf', 'LyricistOf', 'ProducerOf']:
                if G.nodes[edge[1]].get('Node Type') in ['Song', 'Album']:
                    if G.nodes[edge[1]].get('notable'):
                        notable_work_num += 1
        G.nodes[node_id]['notable_work_num'] = notable_work_num
        if notable_work_num > 5:
            G.nodes[node_id]['notable'] = True
        else:
            G.nodes[node_id]['notable'] = False
        print(f"节点 {node_id} 的显著作品数量: {notable_work_num}")


节点 1 的显著作品数量: 2
节点 2 的显著作品数量: 2
节点 3 的显著作品数量: 1
节点 6 的显著作品数量: 1
节点 7 的显著作品数量: 10
节点 8 的显著作品数量: 2
节点 11 的显著作品数量: 3
节点 12 的显著作品数量: 2
节点 13 的显著作品数量: 4
节点 16 的显著作品数量: 1
节点 17 的显著作品数量: 0
节点 18 的显著作品数量: 0
节点 19 的显著作品数量: 1
节点 20 的显著作品数量: 2
节点 27 的显著作品数量: 4
节点 28 的显著作品数量: 1
节点 29 的显著作品数量: 9
节点 30 的显著作品数量: 33
节点 36 的显著作品数量: 1
节点 37 的显著作品数量: 3
节点 40 的显著作品数量: 5
节点 43 的显著作品数量: 6
节点 44 的显著作品数量: 2
节点 45 的显著作品数量: 2
节点 49 的显著作品数量: 1
节点 50 的显著作品数量: 2
节点 51 的显著作品数量: 1
节点 52 的显著作品数量: 6
节点 55 的显著作品数量: 2
节点 56 的显著作品数量: 6
节点 57 的显著作品数量: 2
节点 61 的显著作品数量: 1
节点 62 的显著作品数量: 1
节点 63 的显著作品数量: 1
节点 64 的显著作品数量: 1
节点 65 的显著作品数量: 6
节点 66 的显著作品数量: 1
节点 67 的显著作品数量: 5
节点 72 的显著作品数量: 2
节点 73 的显著作品数量: 1
节点 74 的显著作品数量: 8
节点 75 的显著作品数量: 3
节点 76 的显著作品数量: 6
节点 82 的显著作品数量: 6
节点 83 的显著作品数量: 4
节点 87 的显著作品数量: 2
节点 88 的显著作品数量: 1
节点 89 的显著作品数量: 3
节点 90 的显著作品数量: 6
节点 93 的显著作品数量: 3
节点 94 的显著作品数量: 1
节点 95 的显著作品数量: 1
节点 96 的显著作品数量: 1
节点 97 的显著作品数量: 1
节点 98 的显著作品数量: 3
节点 99 的显著作品数量: 7
节点 103 的显著作品数量: 2
节点 104 的显著作品数量: 2
节点 107 的显著作品数量: 

In [62]:
from networkx.readwrite import node_link_data

# Convert the graph to JSON format
graph_data = node_link_data(G)

# Save the JSON data to a file
output_file = 'graph_processed.json'
with open(output_file, 'w', encoding='utf-8') as f:
    json.dump(graph_data, f, ensure_ascii=False, indent=4)

print(f"Graph has been saved to {output_file}")

Graph has been saved to graph_processed.json
